LiveRamp Quickstart example SQL

 This will help a developer who has the LiveRamp Native Application for Identity installed
 It will perform the following functions
 1. Match a table on HEMS (Assuming you have to HEM tables)
 2. Setup and initiate the LiveRamp Native Identity Resolution
 3. Attempt a match on RampIDs with you and a partner
 4. Setup and initiate the LiveRamp Native Identity Translation
 5. Match using the RampIDs between You, a partner and the Translation Table Output


 If you need additional support reach out to your LiveRamp representative


In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
// **************************************************************************************
// **************************************************************************************
// 1. Start Match Using HEMS
// **************************************************************************************
// **************************************************************************************

-- create the match report on hashed emails
WITH total_table1 AS (
    SELECT COUNT(*) AS total_count1
    FROM "<Your HEM Input Table>"
),
total_table2 AS (
    SELECT COUNT(*) AS total_count2
    FROM "<Partners HEM Input>"
),
matching_rows AS (
    SELECT COUNT(*) AS match_count
    FROM "<Your HEM Input Table>" t1
    INNER JOIN "<Partners HEM Input>" t2 ON t1.hashed_email = t2.hashed_email
)
-- Calculate the match rate
SELECT (match_count::FLOAT / (total_count1)) * 100 AS match_rate_percentage
FROM total_table1, total_table2, matching_rows;


// **************************************************************************************
// **************************************************************************************
// 1. END Match Using HEMS
// **************************************************************************************
// **************************************************************************************


In [ ]:
// **************************************************************************************
// **************************************************************************************
// 2. Start Identity Resolution
// **************************************************************************************
// **************************************************************************************

--Define the role used to grant permissions to native app.
set role_name = 'ACCOUNTADMIN';

--Update this section with the appropriate variables
set customer_db_name = '<your db>';
set customer_schema_name=concat($customer_db_name,'.','<your schema>');
set customer_input_table_name=concat($customer_schema_name,'.','<input Table>');
set customer_meta_table_name=concat($customer_schema_name,'.','<meta table name>');
set customer_metrics_table_name=concat($customer_schema_name,'.','<metrics table>');
set customer_logging_table_name=concat($customer_schema_name,'.','<logging table>');

--name of database where LiveRamp share is loaded
-- V1 set native_app_db_name='LR_APP_SHARE_RESOLUTION_DEMO';
set application_name = '<native application name>';
set native_app_schema_name = 'LR_APP_SCHEMA';
set native_job_schema_name = 'LR_JOB_SCHEMA';
--Create output table for  results (please update)
set output_table_name = '<output table>';


SET final_output = concat($application_name,'.',$native_job_schema_name,'.',$output_table_name);
--replace 

// Defining metadata

--Switch to the database and schema that has your input table for Resolution
USE DATABASE identifier($customer_db_name);
USE schema identifier($customer_schema_name);

--Metadata table schema
create or replace table identifier($customer_meta_table_name) as
select
    TO_VARCHAR(DECRYPT(ENCRYPT('<client_id>', 'HideFromLogs'), 'HideFromLogs'), 'utf-8') as client_id,
    TO_VARCHAR(DECRYPT(ENCRYPT('<client_secret>', 'HideFromLogs'), 'HideFromLogs'), 'utf-8') as client_secret,
    'resolution' as execution_mode,
    'pii' as execution_type,
    parse_json($$
    {
      "name": ["<up to 4 name column names>"],
      "streetAddress": ["<up to 7 address column names>"],
      "city": "<city column>",
      "state": "<state column>",
      "zipCode": "<zipcode column>",
      "phone": "<phone column>",
      "email": "<email column>"
    }
    $$) as target_columns,
    1 as limit;

grant usage on database identifier ($customer_db_name) to application identifier($application_name);
grant usage on schema identifier ($customer_schema_name) to application identifier($application_name);
grant select on table identifier ($customer_input_table_name) to application identifier($application_name);
grant select on table identifier ($customer_meta_table_name) to application identifier($application_name);


use database identifier ($application_name);
use schema lr_app_schema;

--Identity Resolution Call
call lr_resolution_and_transcoding(
	$customer_input_table_name,
	$customer_meta_table_name,
	$output_table_name,       
    $customer_logging_table_name,
	$customer_metrics_table_name
);


select * from "<your logging table>"
order by ts desc
limit 120;

select * from "<your metrics table>"
order by event_time desc
limit 30;


call check_for_output(
	$output_table_name
);

// **************************************************************************************
// **************************************************************************************
// 2. End Identity Resolution
// **************************************************************************************
// **************************************************************************************


In [ ]:
// **************************************************************************************
// **************************************************************************************
// 3. Start Match before Transcoding
// **************************************************************************************
// **************************************************************************************

-- create the match report
WITH total_table1 AS (
    SELECT COUNT(*) AS total_count1
    FROM "<Your RampID enabled table>"
),
total_table2 AS (
    SELECT COUNT(*) AS total_count2
    FROM "<Partners RampID Enabled Table>"
),
matching_rows AS (
    SELECT COUNT(*) AS match_count
    FROM "<Your RampID enabled table>" t1
    INNER JOIN "<Partners RampID Enabled Table>" t2 ON t1.rampid = t2.rampid
)
-- Calculate the match rate
SELECT (match_count::FLOAT / (total_count1)) * 100 AS match_rate_percentage
FROM total_table1, total_table2, matching_rows;
// **************************************************************************************
// **************************************************************************************
// 3. END Match before Transcoding
// **************************************************************************************
// **************************************************************************************


In [ ]:
// **************************************************************************************
// **************************************************************************************
// 4. Start A Transcoding setup
// **************************************************************************************
// **************************************************************************************


CREATE OR REPLACE TABLE "<input table>"
AS SELECT RAMPID , '<Partner Domain>' AS TARGET_DOMAIN , 'RampID' AS TARGET_TYPE FROM "<output of Identity Resolution";

--Define the role used to grant permissions to native app.
set role_name = 'ACCOUNTADMIN';

--Update this section with the appropriate variables
set customer_db_name = '<your db>';
set customer_schema_name=concat($customer_db_name,'.','<your schema>');
set customer_input_table_name=concat($customer_schema_name,'.','<input Table>');
set customer_meta_table_name=concat($customer_schema_name,'.','<meta table name>');
set customer_metrics_table_name=concat($customer_schema_name,'.','<metrics table>');
set customer_logging_table_name=concat($customer_schema_name,'.','<logging table>');

--name of database where LiveRamp share is loaded
-- V1 set native_app_db_name='LR_APP_SHARE_RESOLUTION_DEMO';
set application_name = '<native application name>';
set native_app_schema_name = 'LR_APP_SCHEMA';
set native_job_schema_name = 'LR_JOB_SCHEMA';
--Create output table for  results (please update)
set output_table_name = '<output table>';


create or replace table identifier($customer_meta_table_name) as
select
    TO_VARCHAR(DECRYPT(ENCRYPT('<client_id>', 'HideFromLogs'), 'HideFromLogs'), 'utf-8') as client_id,
    TO_VARCHAR(DECRYPT(ENCRYPT('<client_secret>', 'HideFromLogs'), 'HideFromLogs'), 'utf-8') as client_secret,
    'transcoding' as execution_mode,
    'transcoding' as execution_type,
    '<column to be translated>' as target_column,
    '<column containing target domain>' as target_domain_column,
    '<column containing target type>' as target_type_column;
    
--Switch to Native App database and schema
USE DATABASE identifier($customer_db_name);
USE schema identifier($customer_schema_name);


-- Grant permissions
grant usage on database identifier ($customer_db_name) to application identifier($application_name);
grant usage on schema identifier ($customer_schema_name) to application identifier($application_name);
grant select on table identifier ($customer_input_table_name) to application identifier($application_name);
grant select on table identifier ($customer_meta_table_name) to application identifier($application_name);


use database identifier ($application_name);
use schema lr_app_schema;

--Identity Resolution Call
call lr_resolution_and_transcoding(
	$customer_input_table_name,
	$customer_meta_table_name,
	$output_table_name,       
    $customer_logging_table_name,
	$customer_metrics_table_name
);

----Checking for Output
call check_for_output(
	$output_table_name
);


// **************************************************************************************
// **************************************************************************************
// 4. END Transcoding A setup
// **************************************************************************************
// **************************************************************************************


In [ ]:
// **************************************************************************************
// **************************************************************************************
// 5. Start Match Rate on RampID
// **************************************************************************************
// **************************************************************************************


WITH total_table1 AS (
    SELECT COUNT(*) AS total_count1
    FROM "<Your input RampID enabled Table>"
),
total_table2 AS (
    SELECT COUNT(*) AS total_count2
    FROM "<Partners RampID Enabled Table>"
),
matching_rows AS (
    SELECT COUNT(*) AS match_count
    FROM "<Your input RampID enabled Table>" t1
    INNER JOIN "<Translation Output Table>" t3
        ON t1.RampID = t3.RampID
    INNER JOIN "<Partners RampID Enabled Table>" t2
        ON t3.TRANSCODED_IDENTIFIER = t2.RampID
)
-- Calculate the match rate
SELECT (match_count::FLOAT / (total_count1)) * 100 AS match_rate_percentage
FROM total_table1, total_table2, matching_rows;

// **************************************************************************************
// **************************************************************************************
// 5. END Transcoding Match Rate on RampID
// **************************************************************************************
// **************************************************************************************
